# Tools

In [1]:
from ipynb.fs.full.koselleck import *

## Source code

In [2]:
def display_source(code):
    import IPython
    def _jupyterlab_repr_html_(self):
        from pygments import highlight
        from pygments.formatters import HtmlFormatter

        fmt = HtmlFormatter()
        style = "<style>{}\n{}\n{}\n</style>".format(
            fmt.get_style_defs(".output_html"),
            fmt.get_style_defs(".jp-RenderedHTML"),
            '.jp-RenderedHTML span { font-size:0.8em }'
        )
        return style + highlight(self.data, self._get_lexer(), fmt)

    # Replace _repr_html_ with our own version that adds the 'jp-RenderedHTML' class
    # in addition to 'output_html'.
    IPython.display.Code._repr_html_ = _jupyterlab_repr_html_
    return IPython.display.Code(data=code, language="python3")

In [3]:
display_source("print('Hello world')")

print('Hello world')

In [4]:
# Source a function
def src(x):
    from IPython.display import Code,display
    if type(x)!=str:
        import inspect
        x=inspect.getsource(x)
    display(display_source(x))

In [5]:
src(src)

def src(x):
    from IPython.display import Code,display
    if type(x)!=str:
        import inspect
        x=inspect.getsource(x)
    display(display_source(x))

In [6]:
def get_numlines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [7]:
get_numlines('tools.ipynb')

1333

## Figures

In [1]:
def upfig(fnfn,uproot=UPROOT):
    ofnfn=os.path.join(uproot,os.path.basename(fnfn))
    os.system(f'dbu upload {fnfn} {ofnfn}')
    return os.system(f'dbu share {ofnfn}')

NameError: name 'UPROOT' is not defined

In [9]:
from IPython.display import Image as iImage
def print_img(fn):
    #return iImage(filename=fn)
    with open(fn,'rb') as f:
        display(widgets.Image(value=f.read(),format='png'))

In [10]:
# def df_to_png(df,ofn='fig.dataframe.png',style=None):
#     import imgkit
    
#     STYLE="""
#         table { border:1px dotted silver; }
#         td,th { font-family: "Baskerville", "Libre Baskerville", "Georgia", serif; }
#         tbody tr:nth-child(even)   { background-color:#eee; }
#         tbody tr:nth-child(odd)    { background-color:#fff; }
#         thead { background-color:#ddd; }
#     """
#     style = STYLE if not style else style
#     style=f'<style type="text/css">{style}</style>'    
#     ohtml=df.to_html()
#     o='<html><body>'+style + ohtml+'</body></html>'
#     imgkit.from_string(o, ofn, options={'--quiet':''})
#     print_img(ofn)
#     return ofn

## Periodizing

In [11]:
def periodize(y,ybin=5):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [12]:
def periodize_bystep(y,ybin,ymin,ymax=10000):
    if y<ymin or y>ymax: return ''
    ystep=ymin
    for ystep2 in range(ymin,ymax+ybin,ybin):
        if y>=ystep and y<ystep2:
            return f'{ystep}-{ystep2}'
        ystep=ystep2
    return ''

In [13]:
def get_periods_bystep(ymin=YMIN, ymax=YMAX, ybin=YEARBIN):
    steps=list(range(ymin,ymax+ybin,ybin))
    step1=steps[0]
    l=[]
    for step2 in steps[1:]:
        o=f'{step1}-{step2}'
        l+=[o]
        step1=step2
    return l

# get_periods_bystep(ymin=1700, ymax=1900, ybin=40)

In [14]:
periodize(1848)

'1845-1850'

In [15]:
## Random
def periodize_sattelzeit(y,use_dates=True):
    if 1700<=y<1770: return '1700-1770' if use_dates else 'Before'
    if 1770<=y<1830: return '1770-1830' if use_dates else 'During'
    if 1830<=y<1900: return '1830-1900' if use_dates else 'After'

def periodize_sattelzeit_binary(y):
    if 1700<=y<1770: return '1700-1770'
    if 1830<=y<1900: return '1830-1900'

In [16]:
periodize_sattelzeit(1789,use_dates=False)

'During'

## Files

In [17]:
def read_df(ifn,key='',**attrs):
    if not os.path.exists(ifn): return
    import pandas as pd
    ext = os.path.splitext(ifn.replace('.gz',''))[-1][1:]
    if ext=='csv':
        return pd.read_csv(ifn,**attrs)
    elif ext in {'xls','xlsx'}:
        return pd.read_excel(ifn,**attrs)
    elif ext in {'txt','tsv'}:
        return pd.read_csv(ifn,sep='\t',**attrs)
    elif ext=='ft':
        return pd.read_feather(ifn,**attrs)
    elif ext=='pkl':
        return pd.read_pickle(ifn,**attrs)
    elif ext=='h5':
        return pd.read_hdf(ifn, key=key,**attrs)
    else:
        raise Exception(f'[save_df()] What kind of df is this: {ifn}')


## Tokenizers

In [18]:

def tokenize_fast(line,lower=True,incl_punct=False):
    import re
    from string import punctuation
    line=line.lower() if lower else line
    tokens = re.findall(
        r".*?[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+",
        # r'\w+',
        line
    )
    if not incl_punct: tokens = [w.strip(punctuation) for w in tokens]
    tokens = [w for w in tokens if w]
    return tokens

# set tokenizer
tknz=tokenize_fast

In [19]:
tknz('ThiS,is,a sentence. !! ---hello')

['this', 'is', 'a', 'sentence', 'hello']

In [22]:
def to_words(word_or_words):
    return tokenize_fast(word_or_words) if type(word_or_words)==str else word_or_words

In [ ]:
def get_words_corpora_periods(
        word_or_words, period_or_periods=None, corpus_or_corpora=DEFAULT_CORPUS,
        ybin=YBIN_WDIST, ymin=YMIN_WDIST, ymax=YMAX_WDIST):
    words=to_words(word_or_words)
    corpora=to_words(corpus_or_corpora)
    periods=to_words(period_or_periods) if period_or_periods else get_periods_bystep(
        ymin=ymin,ymax=ymax,ybin=ybin
    )
    return words,corpora,periods

## Figures

In [21]:
def start_fig(data=None, theme='minimal',text_size=8, figure_size=(8,8), **aesd):
    p9.options.figure_size=figure_size
    p9.options.dpi=600
    fig=p9.ggplot(p9.aes(**aesd), data=data)
    fig+=getattr(p9,f'theme_{theme}')()
    fig+=p9.theme(
        text=p9.element_text(size=text_size),
#         plot_background=p9.element_rect(fill='white')
    )
    return fig

In [21]:
# read_df('../data/data.model.paths.default.pkl')

## Markdown

In [1]:
from IPython.display import display,Markdown
import pandas as pd

def printm(x):
    if type(x)==pd.DataFrame: x=x.to_markdown()
    display(Markdown(x))

def markdwn(x):
    html = markdown.markdown(x,extensions=['tables'])
    return HTML(html)

## Googlesheets

In [26]:
def get_sheet(sh_id=GSPREAD_NAME):
    return Spread(sh_id) if type(sh_id)==str else sh_id

In [33]:
# sh=get_sheet()

In [29]:
def write_sheet(df,sh_or_id=GSPREAD_NAME,index=True,replace=True):
    sh = get_sheet(sh_or_id)
    sh.df_to_sheet(df,index=index,replace=replace)

In [1]:
def read_sheet(sh_or_id=GSPREAD_NAME):
    sh = get_sheet(sh_or_id)
    dfanno=sh.sheet_to_df()
    dfanno['rank']=dfanno['rank'].replace({'':-1}).apply(int)
    dfanno=dfanno.sort_values('rank')
    return dfanno

NameError: name 'GSPREAD_NAME' is not defined

In [14]:

# def classify_vector_change(row,perc_threshold=75,z_threshold=1,z_threshold2=.5):
#     name=row.vector
#     ns=str(name).split('.')[0].split('-')
#     n1,n2=ns[0],ns[-1]
#     # res='~'#f'{n1}~{n2}'
#     res='+' if row.mean_diff>0 else '-'
#     if row.mean_diff_p<0.05 and row.perc>perc_threshold:
#         if row.mean_diff>0:
#             ## increases
#             # decreases
#             if row.mean2<-z_threshold and row.mean1<-z_threshold:
#                 # still over the line for conc
#                 res=f'{n2}-'
#             elif row.mean2>z_threshold:
#                 # we're over the line for pos
#                 if row.mean1>z_threshold:
#                     # already was though
#                     res=f'{n1}+'                    
#                 elif row.mean1<-z_threshold:
#                     # jumped all the way from neg to pos!
#                     res=f'{n1}+++' 
#                 elif row.mean1<z_threshold2:
#                     # jumped part of the way to become new
#                     res=f'{n1}++'
#                 else:
#                     res=f'{n1}+'
#             elif row.mean1<-z_threshold:
#                 # really this is a loss of the prior
#                 res=f'{n2}--'
#             else:
#                 res=f'{n1}+'
#         else:
#             # decreases
#             if row.mean2>z_threshold and row.mean1>z_threshold:
#                 # still over the line for abs
#                 res=f'{n1}-'
#             elif row.mean2<-z_threshold:
#                 # over the line for neg
#                 if row.mean1<-z_threshold:
#                     # already was tho
#                     res=f'{n2}+'
#                 elif row.mean1>z_threshold:
#                     # jumped all the way from pos to neg
#                     res=f'{n2}+++'
#                 elif row.mean1>-z_threshold2:
#                     # jumped part of the way to become new
#                     res=f'{n2}++'
#                 else:
#                     res=f'{n2}+'
#             elif row.mean1>z_threshold:
#                 # really this is a loss of the prior
#                 res=f'{n1}--'
#             else:
#                 res=f'{n2}+'
#     return res

# DFCC=None
# DFCC_CACHE_FN=os.path.join(PATH_DATA,'data.classed_changes.pkl')
# def get_classed_changes(cache=True):
#     global DFCC
#     if DFCC is None:
#         if cache and os.path.exists(DFCC_CACHE_FN):
#             df=pd.read_pickle(DFCC_CACHE_FN)
#         else:
#             df=get_dfchange_from_decade_data(wide=False).reset_index()
#             df['perc']=df.mean_diff_t_abs.apply(lambda x: percentileofscore(df.mean_diff_t_abs, x))
#             dfchsz=get_dfchange_from_sattelzeit_models().reset_index()
#             df['change']=df.progress_apply(classify_vector_change,axis=1)
#             dfchsz['change']=dfchsz['class_change']
#             df=df.append(dfchsz)
#             startcols=['word','vector','change']
#             df=df[startcols + [c for c in df.columns if c not in set(startcols)]]
#             df['mean_diff_abs']=df['mean_diff'].apply(abs)
#             df['change_rank']=[
#                 x.count('+') + x.count('-') if len(x)>1 else 0
#                 for x in df.change
#             ]
#             if cache: df.to_pickle(DFCC_CACHE_FN)
#         DFCC=df
#     return DFCC.sort_values(['change_rank','mean_diff_abs'],ascending=False)



# def get_dfpiv(key, df=None, fn=FN_VECTOR_SCORES_RUNS, ymin=1700, ymax=1900, words=None, z=False, axis=1):
#     if df is None: df=pd.read_pickle(fn)
#     pwdf=df.groupby(['period','word']).mean().reset_index()
#     pwdf['period']=[int(p[:4]) for p in pwdf.period]
#     pwdf=pwdf.query(f'{ymin}<=period<{ymax}')
#     pwdfpiv=pwdf.pivot('word','period',key)
#     if words: pwdfpiv=pwdfpiv.reset_index()[pwdfpiv.reset_index().word.isin(set(words))].set_index('word')
#     if z: pwdfpiv=to_z(pwdfpiv,axis=axis)
#     return pwdfpiv
    

# def get_dfpiv_abs(key=FIELD_ABS_KEY,**y):
#     return get_dfpiv(key,**y)
    
# def get_pathdf_models_bydecade(ymin=1700,ymax=1900):
#     return get_pathdf_models().query(f'period_len==10 & {ymin}<=period_start<{ymax}')
# def get_pathdf_models_byyear(ymin=1700,ymax=1900):
#     return get_pathdf_models().query(f'period_len==2 & {ymin}<=period_start<{ymax}')




# DFPKG=None
# def get_df_package():
#     global DFPKG
#     if DFPKG is not None: return DFPKG
#         # load decade level data
#     dfpiv_abs,dfpiv_freq = get_decade_level_data()
#     # load diff data by run
#     dfruns=pd.read_csv(FN_CHANGE_RUNS).set_index('word')
#     dfruns_dec=pd.read_pickle(FN_VECTOR_SCORES_RUNS)
#     # Load diffdata avg
#     dfchange=get_dfchange()
#     DFPKG=(dfchange,dfruns,dfpiv_abs,dfpiv_freq,dfruns_dec,get_dfpiv_ambig())
#     return DFPKG

# def get_dfpiv_ambig(fn=FN_AMBIGUITY,z=True):
#     ambdf=pd.read_csv(FN_AMBIGUITY)
#     odf=ambdf.groupby('period').mean()
#     odf.index=[int(p[:4]) for p in odf.index]
#     odf=odf.T
#     if z: odf=to_z(odf)
#     return odf

# def get_dfpiv_freq(words=None,ymin=1700,ymax=1900,z=False):
#     df=pd.read_csv(FN_FREQ_DEC_MODELS).set_index('word')
#     df.columns=[int(x[:4]) for x in df.columns]
#     df=df[[y for y in df.columns if y>=ymin and y<ymax]]
#     if words: df=df.reset_index()[df.reset_index().word.isin(set(words))].set_index('word')
#     if z: df=to_z(df, axis=0)
#     return df

# def get_decade_level_data(words=None,z=True):
#     if os.path.exists(FN_DATA_CACHE_DEC):
#         with open(FN_DATA_CACHE_DEC,'rb') as f:
#             return pickle.load(f)
    
#     # load decade level data
#     if words is None: words=get_valid_words()
#     dfpiv_abs=get_dfpiv_abs(words=words,z=z)#.dropna()
#     dfpiv_freq=get_dfpiv_freq(words=words,z=z)#.dropna()
#     o=dfpiv_abs,dfpiv_freq
#     with open(FN_DATA_CACHE_DEC,'wb') as of:
#         pickle.dump(o,of)
#     return o




# VECNAMES=None
# def get_vector_names():
#     global VECNAMES
#     if not VECNAMES: VECNAMES=list(pd.read_csv(FN_VECTOR_SCORES_DIFFMEANS).vector.unique())
#     return VECNAMES

# def classify_abstractness(row,perc_threshold=70,dist_threshold=0.5):
#     # if row.score_diff_p_abstractness<=0.05 and row.dist_abstractness>=dist_threshold:
#     if row.score_diff_p_abstractness<=0.05 and row.perc_abstractness>=perc_threshold:
#         return '+Abstract' if row.score2_abstractness>row.score1_abstractness else '+Concrete'
#     return 'Abs~Conc'
# def classify_change(row,perc_threshold=70,dist_threshold=0.5):
#     if row.is_clean_noiseaware:
#         if row.perc_local>=perc_threshold:
#         # if row.dist_local>=dist_threshold:
#             return '+Changed'
#     else:
#         return '~Noisy'
#     return '-Changed'


# def get_dfchange_from_sattelzeit_models(words=None):
#     # Load collective difference data
#     dfchange=pd.read_csv(FN_CHANGE_RUNS_AVG).set_index('word').sort_values('rank')
#     dfchange['class_abs']=dfchange.apply(classify_abstractness,1)
#     dfchange['class_change']=dfchange.apply(classify_change,1)
#     dfchange['class_signif']=[(x!='Abs~Conc' or y=='+Changed') for x,y in zip(dfchange.class_abs, dfchange.class_change)]
#     dfchange['class']=[f'{x} {y}' for x,y in zip(dfchange.class_abs, dfchange.class_change)]
    
#     # filter?
#     return dfchange.sort_values('rank')
#     #.query('class_abs!="Abs~Conc" | class_change=="+Changed"')
#     # return dfchange

# def get_dfchange_from_decade_data(fn=FN_VECTOR_SCORES_DIFFMEANS,wide=True):
#     idf=pd.read_csv(fn).set_index(['vector','word'])
#     if not wide: return idf
#     df=None
#     for vecname,gdf in idf.groupby('vector'):
#         gdf=gdf.reset_index().drop('vector',1).set_index('word')
#         gdf.columns=[c+'_'+vecname for c in gdf.columns]
#         df=gdf if df is None else df.join(gdf)
    
#     for c in df.columns:
#         if c.startswith('mean_diff_t_abs'):
#             df['perc_'+c]=df[c].apply(lambda x: percentileofscore(df[c],x))
    
#     return df

# def get_dfchange(words=None):
#     odf=get_dfchange_from_sattelzeit_models().join(get_dfchange_from_decade_data(), how='outer')
#     if words: odf=odf.reset_index()[odf.reset_index().word.isin(set(words))].set_index('word')
#     return odf

# def get_dfchange_simple(dfchange=None, col='mean_diff'):
#     if dfchange is None: dfchange=get_dfchange()
#     vecs=get_vector_names()
#     odf=dfchange[[
#         c
#         for c in dfchange.columns
#         if c.startswith(col+'_')
#         and c[len(col)+1:] in set(vecs)
#     ]]
#     odf.columns=[c[len(col)+1:] for c in odf.columns]
#     return odf


# def show_change_table(dfchange,gby=['class_abs','class_change']):
#     pd.options.display.max_colwidth=100
#     dfchange_f=dfchange.query('class_signif==True')
#     for gname,gdf in dfchange_f.groupby(gby[0]):
#         printm('## '+gname)
#         gdfx=pd.DataFrame()
#         for gname2,gdf2 in gdf.groupby(gby[1]):
#     #         printm('### '+gname2)
#             o=', '.join(gdf2.sort_values('rank_local').index)
#     #         printm()
#             gdfx[gname2]=[o]
#         printm(gdfx.to_markdown())
#     #     break





# C=None
# def get_corpus():
#     global C
#     if C is None:
#         C=lltk.load(CNAME)
#     return C

# def load_model_row(row,**y):
#     return load_model(row.path, row.path_vocab,**y)



# #### Misc
# def to_z(pivdf,axis=1,progress=False):
#     pivdf=pivdf.T if not axis else pivdf
#     pivdfz=pd.DataFrame(index=pivdf.index, columns=pivdf.columns)
#     for c in (tqdm(pivdf.columns) if progress else pivdf.columns):
#         pivdfz[c]=(pivdf[c] - pivdf[c].mean()) / pivdf[c].std()
#     return pivdfz.T if not axis else pivdfz


# def start_fig(data=None, theme='minimal',text_size=8, figure_size=(8,8), **aesd):
#     p9.options.figure_size=figure_size
#     p9.options.dpi=600
#     fig=p9.ggplot(p9.aes(**aesd), data=data)
#     fig+=getattr(p9,f'theme_{theme}')()
#     fig+=p9.theme(
#         text=p9.element_text(size=text_size),
#         plot_background=p9.element_rect(fill='white')
#     )
#     return fig
    
    
    
    

# def get_rate_of_change_data(key='dist_local'):
#     df=pd.read_pickle('data/data.semantic_change_over_decades.1run.v3.pkl')
#     df['period']=[f'{x[:4]}s->{y[:4]}s' for x,y in zip(df.period1,df.period2)]
#     df['period_int1']=[int(y[:4]) for y in df.period1]
#     df['period_int2']=[int(y[:4]) for y in df.period2]    
#     return df

# def get_figdf1_rateofchange(df=None,randomize=False):
#     if df is None: df=get_rate_of_change_data()
#     pdf=df.groupby(['period','period1','period2']).mean().reset_index()
#     if randomize:
#         for c in ['period1','period2']:
#             pdf[c]=list(pdf[c].sample(frac=1))
#     pdf2=pd.DataFrame(pdf).rename(columns=dict(
#         period1='period2',
#         period2='period1',
#     ))
#     figdf=pdf.append(pdf2)
#     figdf['period_int1']=figdf.period1.apply(lambda x: int(x[:4]))
#     figdf['period_int2']=figdf.period2.apply(lambda x: int(x[:4]))
#     figdf['perc_local_int']=figdf.dist_local.apply(lambda x: percentileofscore(figdf.dist_local, x)).apply(int)    
#     return figdf



# DFWAS=None
# def get_word_abstractness_scores(cols=['Abs-Conc.Median.C18','Abs-Conc.Median.C19']):
#     global DFWAS
#     if DFWAS is None:
#         DFWAS=get_allnorms()[cols].reset_index().dropna().set_index('word').mean(axis=1)
#     return DFWAS


# def prdz(y,ystart=1710,yend=3000,ystep=40):
#     ln=None
#     for n in range(ystart,yend,ystep):
#         if ln is None: ln=n
#         if y<n: return ln
#         ln=n
        


# def get_novelty_data(ifn=FN_NOVELTY_DATA):
#     allres = pd.read_pickle(ifn).query('foote_novelty!=0.0')
#     allres['is_signif']=[int(x<0.05 or y<0.05)
#                         for x,y in zip(allres.p_peak,allres.p_trough)]
#     allres['foote_size']=allres.foote_size.apply(int)
#     allres['year']=allres.year.apply(int)
#     allres = pd.concat(grp.assign(glen=len(grp)) for i,grp in allres.groupby(['foote_size','year'])).reset_index()
#     allres = pd.concat(
#         grp.sort_values('year').assign(
#             foote_novelty_z=((grp.foote_novelty - grp.foote_novelty.dropna().mean()) / grp.foote_novelty.dropna().std())
#         )#.set_index('year').rolling(rolling,min_periods=min_periods).mean()
#         for i,grp in allres.groupby('foote_size')
#     )
#     return allres


# C=get_corpus()
# logger.remove()
# logger.add(sys.stderr, format="{message}", filter='koselleck', level="INFO")
# # logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
# def log(*x,**y): logger.info(' '.join(str(xx) for xx in x),**y)
    

    
    
    
# UPROOT='/Markdown/Drafts/TheGreatAbstraction/figures/'
# def upfig(fnfn,uproot=UPROOT):
#     ofnfn=os.path.join(uproot,os.path.basename(fnfn))
#     cmd=f'dbu upload {fnfn} {ofnfn}'
#     os.system(cmd)
#     cmd = f'dbu share {ofnfn}'
#     os.system(cmd)
    
    
    
    
# def rsync(ifnfn,ofnfn,flags='-avP'):
#     return runcmd(f'rsync {flags} {ifnfn} {ofnfn}')

# def _rsync_data_from_ember(obj): return rsync(obj[0], obj[1])
# def rsync_data_from_ember(num_proc=1):
#     paths_I_want = [
#         os.path.join(
#             os.path.dirname(path),
#             'dists.pkl'
#         ).split('/ryan/')[-1] for path in get_model_paths_df(PATH_MODELS_BPO).path
#     ]
#     objs = [
#         (f'ryan@ember:{fn}', os.path.join(os.path.expanduser('~'), fn))
#         for fn in paths_I_want
#     ]
#     objs = [(x,y) for x,y in objs if not os.path.exists(y)]
    
#     return pmap(_rsync_data_from_ember, objs, num_proc=num_proc, desc='Rsyncing data from ember')
# def runcmd(cmd,verbose=False):
#     import subprocess
#     print('>>',cmd)
#     result = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT).decode()
#     return result


# def periodize(y,ybin=5):
#     y1=y//ybin*ybin
#     y2=y1+ybin
#     return f'{y1}-{y2}'

In [ ]:
#